In [166]:
import pandas as pd
import numpy as np
import preprocessing
import metrics_experiment
from sklearn.metrics import auc
import metrics_helper
from datetime import timedelta
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import datetime
import warnings
warnings.filterwarnings('ignore')
# Permanently changes the pandas settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [167]:
libre_tests = pd.read_csv('../tests/test-data/hypo_episodes/libre_tests.csv')
dexcom_tests = pd.read_csv('../tests/test-data/hypo_episodes/dexcom_tests.csv')

In [94]:
def calc_episodes(df, hypo, thresh, mins, long_mins=None, lv2=False):
    # Gives a consecutive unique number to each set of consecutive readings below
    # 3.9mmol/L
    if hypo:
        bool_array = df['glc'] < thresh
    else:
        bool_array = df['glc'] > thresh
    unique_num = bool_array.ne(bool_array.shift()).cumsum()
    number_consec = unique_num.map(unique_num.value_counts()).where(bool_array)
    df_unique = pd.DataFrame({'time_rep': df['time'], 'glc_rep':
                            df['glc'], 'unique_number_low': unique_num,
                            'consec_readings_low': number_consec})
    #print(df_unique.tail())
    # Drop any null glucose readings and reset index
    df_unique.dropna(subset=['glc_rep'], inplace=True)
    df_unique.reset_index(inplace=True, drop=True)
    #print(df_unique)
    # Group by the unique number to collapse into episodes, then use min to
    # calculate the minimum glucose for each bout and the start time
    unique_min = df_unique.groupby('unique_number_low').min()
    
    # Use the start time of bouts and periods between bouts to calculate duration
    # of episodes
    #unique_min['diff'] = unique_min.time_rep.diff().shift(-1)
    unique_min['diff'] = df_unique.groupby('unique_number_low').apply(lambda group: calc_diff(group))
    if lv2:
        # Only keep hypos that are 15 mins or longer (smaller than this doesn't count)
        long_hypos = unique_min[(unique_min['diff'] >= timedelta(minutes=long_mins))&(pd.notnull(unique_min['consec_readings_low']))]
        long_hypos = long_hypos.shape[0]

    # Only keep hypos that are 15 mins or longer (smaller than this doesn't count)
    results = unique_min[unique_min['diff'] >= timedelta(minutes=mins)]

    # Fill the consec readings with binary value to show whether they are hypos or
    # the periods between hypos
    results.consec_readings_low = results.consec_readings_low.fillna(-1)
    results['hypo'] = results['consec_readings_low'] > 0

    # Merge any consecutive values left by removal of too-short episodes using
    # a new unique number
    results['unique'] = results['hypo'].ne(results['hypo'].shift()).cumsum()

    # Group by the unique number, select the min values and select relevant columns
    final_results = results.groupby('unique').min()[['time_rep', 'glc_rep', 'hypo', 'diff']]

    # Calculate difference between hypo and non-hypo periods and shift column up to
    # get the final duration of the periods
    final_results['diff2'] = final_results['time_rep'].diff().shift(-1)

    # Drop the non-hypo periods and then drop the hypo column
    final_results = final_results.loc[final_results['hypo'] ==
                                      True].drop(columns=['hypo'])

    # Rename columns
    final_results.columns = ['start_time', 'min_glc', 'initial_duration', 'duration']

    # Fill final hypo with previous duration value in diff col then drop initial
    # duration
    finaL_results = final_results['duration'].fillna(final_results['initial_duration'])
    final_results.drop(columns=['initial_duration'], inplace=True)
    final_results.reset_index(drop=True, inplace=True)
    # Drop the final column if it's less than 15 mins
    final_results = final_results.loc[final_results['duration']>=
                                      timedelta(minutes=mins)]

    # Create new column identifying if the hypo is level 2 (<3mmol/L)
    #final_results['lv2'] = final_results['min_glc'] < lv2_thresh
    
    # Reset index
    final_results.reset_index(drop=True, inplace=True)
    
    # Calculate overview statistics
    number_hypos = final_results.shape[0]

    if lv2:
        if hypo:
            return {'Level 2 hypoglycemic episodes':number_hypos, f'Prolonged level 2 hypoglycemic episodes ({long_mins} mins)':long_hypos}
        else:
            return {'Level 2 hyperglycemic episodes':number_hypos, f'Prolonged level 2 hyperglycemic episodes ({long_mins} mins)':long_hypos}
    else:
        if hypo:
            return {'Total hypoglycemic episodes':number_hypos}
        else:
            return {'Total hyperglycemic episodes':number_hypos}

In [89]:
def number_of_episodes(df, hypo=True, mins=15, long_mins=120, lv1_hypo_thresh=3.9, lv2_hypo_thresh=3, lv1_hyper_thresh=10, lv2_hyper_thresh=13.9):
    '''
    Replacement helper for number of hypos
    '''
    # Set the time column to datetime and sort by it
    df['time'] = pd.to_datetime(df['time'])
    df.sort_values('time', inplace=True)
    #hypo_1 = calc_episodes(df, hypo=True, thresh=lv1_hypo_thresh, mins=mins, lv2=False)
    #hypo_2 = calc_episodes(df, hypo=True, thresh=lv2_hypo_thresh, mins=mins, long_mins=long_mins, lv2=True)
    hyper_1 = calc_episodes(df, hypo=False, thresh=lv1_hyper_thresh, mins=mins, lv2=False)
    #hyper_2 = calc_episodes(df, hypo=False, thresh=lv2_hyper_thresh, mins=mins,  long_mins=long_mins, lv2=True)
    print(hyper_1)
    #print(hyper_2)

    #hypo_1.update(hypo_2)
    #hypo_1.update(hyper_1)
    #hypo_1.update(hyper_2)
    #return hypo_1

In [168]:
df = libre_tests[['time', 'test10']]
df.columns = ['time', 'glc']
df = df.dropna()
#df = df.iloc[0:16]
#number_of_episodes(df)

In [254]:
def calc_diff(group):
    endtime = group.iloc[-1]['time_rep']
    starttime = group.iloc[0]['time_rep']
    diff = endtime-starttime
    return diff

In [255]:
def collapse_bool_array(bool_array):
    unique_num = bool_array.ne(bool_array.shift()).cumsum()
    number_consec = unique_num.map(unique_num.value_counts()).where(bool_array)
    df_unique = pd.DataFrame({'time_rep': df['time'], 'glc_rep':
                            df['glc'], 'unique_number': unique_num,
                            'consec_readings': number_consec})
    df_unique['time_rep'] = pd.to_datetime(df_unique['time_rep'])
    # Drop any null glucose readings and reset index
    df_unique.dropna(subset=['glc_rep'], inplace=True)
    df_unique.reset_index(inplace=True, drop=True)
    diff = df_unique.groupby('unique_number').apply(lambda group: calc_diff(group))
    unique_min = df_unique.groupby('unique_number').min()
    unique_min['diff'] = diff
    #unique_min = unique_min.dropna(subset=['consec_readings_low'])
    # Use the start time of bouts and periods between bouts to calculate duration
    # of episodes
    #unique_min['diff'] = unique_min.time_rep.diff().shift(-1)
    return unique_min

In [267]:
def calc_duration(unique_min, mins):
    #unique_min['diff'] = df_unique.groupby('unique_number_low').apply(lambda group: calc_diff(group))
    # Only keep hypos that are 15 mins or longer (smaller than this doesn't count)
    results = unique_min[unique_min['diff'] >= timedelta(minutes=mins)]

    # Fill the consec readings with binary value to show whether they are hypos or
    # the periods between hypos
    results['consec_readings'] = results['consec_readings'].fillna(-1)
    results['hypo'] = results['consec_readings'] > 0

    # Merge any consecutive values left by removal of too-short episodes using
    # a new unique number
    results['unique'] = results['hypo'].ne(results['hypo'].shift()).cumsum()
    return results

In [268]:
hypo=False
lv2=False
thresh = 10
thresh_lv2 = 13.9
mins=15
if hypo:
    bool_array = df['glc'] < thresh
    bool_array_lv2 = df['glc'] < thresh_lv2
else:
    bool_array = df['glc'] > thresh
    bool_array_lv2 = df['glc'] > thresh_lv2

'''unique_num = bool_array.ne(bool_array.shift()).cumsum()
number_consec = unique_num.map(unique_num.value_counts()).where(bool_array)
df_unique = pd.DataFrame({'time_rep': df['time'], 'glc_rep':
                        df['glc'], 'unique_number': unique_num,
                        'consec_readings': number_consec})
df_unique['time_rep'] = pd.to_datetime(df_unique['time_rep'])
# Drop any null glucose readings and reset index
df_unique.dropna(subset=['glc_rep'], inplace=True)
df_unique.reset_index(inplace=True, drop=True)
diff = df_unique.groupby('unique_number').apply(lambda group: calc_diff(group))
unique_min = df_unique.groupby('unique_number').min()
unique_min['diff'] = diff'''

unique_min = collapse_bool_array(bool_array)
unique_min_lv2 = collapse_bool_array(bool_array_lv2)
results = calc_duration(unique_min, mins)
results_lv2 = calc_duration(unique_min_lv2, mins)


In [269]:
unique_min

,time_rep,glc_rep,consec_readings,diff
unique_number,,,,
1,2018-05-21 23:50:00,11.0,1.0,0 days 00:00:00
2,2018-05-22 00:05:00,9.0,NaN,0 days 00:00:00
3,2018-05-22 00:20:00,11.0,2.0,0 days 00:15:00
4,2018-05-22 00:50:00,6.7,NaN,0 days 00:00:00
5,2018-05-22 01:05:00,11.0,1.0,0 days 00:00:00
6,2018-05-22 01:20:00,6.8,NaN,0 days 00:00:00
7,2018-05-22 01:35:00,11.0,2.0,0 days 00:15:00
8,2018-05-22 02:05:00,8.8,NaN,0 days 00:15:00
9,2018-05-22 02:35:00,10.1,4.0,0 days 00:45:00


In [270]:
results

,time_rep,glc_rep,consec_readings,diff,hypo,unique
unique_number,,,,,,
3,2018-05-22 00:20:00,11.0,2.0,0 days 00:15:00,True,1
7,2018-05-22 01:35:00,11.0,2.0,0 days 00:15:00,True,1
8,2018-05-22 02:05:00,8.8,-1.0,0 days 00:15:00,False,2
9,2018-05-22 02:35:00,10.1,4.0,0 days 00:45:00,True,3
11,2018-05-22 03:50:00,10.2,2.0,0 days 00:15:00,True,3
12,2018-05-22 04:20:00,6.2,-1.0,0 days 00:15:00,False,4


In [271]:
unique_min_lv2

,time_rep,glc_rep,consec_readings,diff
unique_number,,,,
1,2018-05-21 23:50:00,6.7,NaN,0 days 02:45:00
2,2018-05-22 02:50:00,14.0,3.0,0 days 00:30:00
3,2018-05-22 03:35:00,6.2,NaN,0 days 01:15:00


In [263]:
results

,time_rep,glc_rep,consec_readings,diff,hypo,unique
unique_number,,,,,,
9,2018-05-22 02:35:00,10.1,4.0,0 days 00:45:00,True,1


In [262]:
diff

unique_number
1    0 days 00:00:00
2    0 days 00:00:00
3    0 days 00:15:00
4    0 days 00:00:00
5    0 days 00:00:00
6    0 days 00:00:00
7    0 days 00:15:00
8    0 days 00:15:00
9    0 days 00:45:00
10   0 days 00:00:00
11   0 days 00:15:00
12   0 days 00:15:00
13   0 days 00:00:00
dtype: timedelta64[ns]

In [221]:
results

,time_rep,glc_rep,consec_readings_low,diff,hypo,unique
unique_number_low,,,,,,
3,2018-05-22 00:20:00,11.0,2.0,0 days 00:30:00,True,1
7,2018-05-22 01:35:00,11.0,2.0,0 days 00:30:00,True,1
8,2018-05-22 02:05:00,8.8,-1.0,0 days 00:30:00,False,2
9,2018-05-22 02:35:00,10.1,4.0,0 days 01:00:00,True,3
11,2018-05-22 03:50:00,10.2,2.0,0 days 00:30:00,True,3
12,2018-05-22 04:20:00,6.2,-1.0,0 days 00:30:00,False,4


In [203]:
# Group by the unique number, select the min values and select relevant columns
results_grouped = results.groupby('unique').min()[['time_rep', 'glc_rep', 'hypo', 'diff']]
results_grouped['diff2'] = results_grouped['time_rep'].diff().shift(-1)
# Drop the non-hypo periods and then drop the hypo column
final_results = results_grouped.loc[results_grouped['hypo'] ==
                                    True].drop(columns=['hypo'])
                                    # Rename columns
final_results.columns = ['start_time', 'min_glc', 'initial_duration', 'duration']

# Fill final hypo with previous duration value in diff col then drop initial
# duration
final_results['duration'] = final_results['duration'].fillna(final_results['initial_duration'])
final_results.drop(columns=['initial_duration'], inplace=True)
# Drop the final column if it's less than 15 mins
final_results = final_results.loc[final_results['duration']>=
                                    timedelta(minutes=mins)]

# Create new column identifying if the hypo is level 2 (<3mmol/L)
#final_results['lv2'] = final_results['min_glc'] < lv2_thresh

# Reset index
final_results.reset_index(drop=True, inplace=True)

# Calculate overview statistics
number_hypos = final_results.shape[0]

In [204]:
results_grouped

,time_rep,glc_rep,hypo,diff,diff2
unique,,,,,
1,2018-05-22 02:35:00,10.1,True,0 days 00:45:00,NaT


In [202]:
final_results

,start_time,min_glc,duration
0,2018-05-22 02:35:00,10.1,0 days 00:45:00


In [162]:
final_results['end_time'] = final_results['start_time']+final_results['duration']

In [163]:
final_results

,start_time,min_glc,duration,end_time
0,2018-05-22 00:20:00,11.0,0 days 01:45:00,2018-05-22 02:05:00
1,2018-05-22 02:35:00,10.1,0 days 01:45:00,2018-05-22 04:20:00


In [161]:
number_hypos

2

In [138]:
final_results['duration'].fillna(final_results['initial_duration'])

unique
2   0 days 01:45:00
4   0 days 00:30:00
Name: duration, dtype: timedelta64[ns]

In [148]:
final_results

,start_time,min_glc,initial_duration,duration
unique,,,,
2,2018-05-22 00:50:00,11.0,0 days 00:30:00,0 days 01:45:00
4,2018-05-22 03:05:00,10.1,0 days 00:30:00,0 days 00:30:00


In [127]:
# Group by the unique number, select the min values and select relevant columns
final_results = results.groupby('unique').min()[['time_rep', 'glc_rep', 'hypo', 'diff']]

# Calculate difference between hypo and non-hypo periods and shift column up to
# get the final duration of the periods
final_results['diff2'] = final_results['time_rep'].diff().shift(-1)

# Drop the non-hypo periods and then drop the hypo column
final_results = final_results.loc[final_results['hypo'] ==
                                    True].drop(columns=['hypo'])

# Rename columns
final_results.columns = ['start_time', 'min_glc', 'initial_duration', 'duration']

# Fill final hypo with previous duration value in diff col then drop initial
# duration
finaL_results = final_results['duration'].fillna(final_results['initial_duration'])
final_results.drop(columns=['initial_duration'], inplace=True)
final_results.reset_index(drop=True, inplace=True)
# Drop the final column if it's less than 15 mins
final_results = final_results.loc[final_results['duration']>=
                                    timedelta(minutes=mins)]

# Create new column identifying if the hypo is level 2 (<3mmol/L)
#final_results['lv2'] = final_results['min_glc'] < lv2_thresh

# Reset index
final_results.reset_index(drop=True, inplace=True)

# Calculate overview statistics
number_hypos = final_results.shape[0]

In [128]:
number_hypos

1

In [117]:
unique_min.dropna(subset=['consec_readings_low'])

,time_rep,glc_rep,consec_readings_low,diff,diff2
unique_number_low,,,,,
2,2018-05-22 00:20:00,11.0,1.0,0 days 00:00:00,0 days 00:30:00
4,2018-05-22 00:50:00,11.0,2.0,0 days 00:15:00,0 days 00:15:00
6,2018-05-22 03:05:00,10.1,4.0,0 days 00:45:00,0 days 01:45:00
8,2018-05-22 04:20:00,10.2,2.0,0 days 00:15:00,0 days 00:15:00


In [107]:
unique_min.dropna(subset=['consec_readings_low'])

,time_rep,glc_rep,consec_readings_low,diff,diff2
unique_number_low,,,,,
2,2018-05-22 03:05:00,10.1,3.0,0 days 00:30:00,0 days 03:15:00
4,2018-05-22 04:20:00,10.2,2.0,0 days 00:15:00,0 days 00:15:00


In [32]:
results = []
for i in libre_tests.columns.drop('time'):
    df = libre_tests[['time', i]]
    df.columns = ['time', 'glc']
    df = df.dropna()
    metrics = number_of_episodes(df)
    results.append(metrics)
results = pd.DataFrame(results)

Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []
Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []
Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []
Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []


C:\Users\cr591\AppData\Local\Temp\ipykernel_8012\3577298277.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results.consec_readings_low = results.consec_readings_low.fillna(-1)
C:\Users\cr591\AppData\Local\Temp\ipykernel_8012\3577298277.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['hypo'] = results['consec_readings_low'] > 0
C:\Users\cr591\AppData\Local\Temp\ipykernel_8012\3577298277.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []
Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []
Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []
Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []
Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []
Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []
Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []
Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []


C:\Users\cr591\AppData\Local\Temp\ipykernel_8012\3577298277.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results.consec_readings_low = results.consec_readings_low.fillna(-1)
C:\Users\cr591\AppData\Local\Temp\ipykernel_8012\3577298277.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['hypo'] = results['consec_readings_low'] > 0
C:\Users\cr591\AppData\Local\Temp\ipykernel_8012\3577298277.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []
Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []
Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []
Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []
Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []
Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []
Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []
Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []
Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []
Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []
Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []
Empty DataFrame
Columns: [time_rep, glc_rep, consec_readings_low, diff]
Index: []


C:\Users\cr591\AppData\Local\Temp\ipykernel_8012\3577298277.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results.consec_readings_low = results.consec_readings_low.fillna(-1)
C:\Users\cr591\AppData\Local\Temp\ipykernel_8012\3577298277.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['hypo'] = results['consec_readings_low'] > 0
C:\Users\cr591\AppData\Local\Temp\ipykernel_8012\3577298277.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

In [33]:
results

,Total hypoglycemic episodes,Level 2 hypoglycemic episodes,Prolonged level 2 hypoglycemic episodes (120 mins),Total hyperglycemic episodes,Level 2 hyperglycemic episodes,Prolonged level 2 hyperglycemic episodes (120 mins)
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,1,1,0,0,0,0
3,1,0,0,0,0,0
4,1,1,0,0,0,0
5,2,1,0,0,0,0
6,0,0,0,1,0,0
7,0,0,0,1,1,0
8,0,0,0,1,1,0
9,0,0,0,0,0,0
